# Simple Epidemic Model of H5N1 Spillover from Cows

This is a simple example of a multi-population/species model inspired by the ongoing (as of Spring 2024) outbreak of H5N1 in cows. The basic idea is to have an ongoing outbreak in cattle where there is migration/movement between farms. The model also has population of farm workers who can be directly (if rarely) infected from cattle and may spread it within itself, and a general population that can be infected from the farmworkers and spread it within itself. 

First imports and setup

In [2]:
import tabularepimdl as tepi

import pandas as pd
import numpy as np

## Defining the population.

To keep things simple we will create a population with 3 locations to start. Each location is assumed to contain some sort of dairy operation with farm workers and a connected general population. To represent this we will have a `Location` column with a location number, a `Population` model that can take on values `cow`, `farm_workers`, or `gen_pop` (general population). Each of these populations will have an infection state (`InfState`) which takes on the typical values of `S`, `I` and `R`. 

For starters let's assume we have a moderate sized breeding/cattle raising operation in a rural area that sends cattle to two other milking operations. One of these operations large and rural (i.e., lots of cows and farm workers, low general population), while the other is small but more closely linked to a moderately sized urban population.

For now no one will be infected (we will add an infection process from an unrepresented bird species later).

In [9]:
populations = pd.DataFrame({
    'Location' : [1]*3 + [2]*3 + [3]*3,
    'Population': pd.Categorical(['cow','farm_worker','gen_pop' ]*3),
    'InfState': pd.Categorical(['S']*9, categories=['S','I','R']),
    'N' : [400,20,5000, #breeding op
           1000, 20, 5000, #large herd
           50, 5, 50000 ], #Small herd
    'T':0
})

print(populations)

   Location   Population InfState      N  T
0         1          cow        S    400  0
1         1  farm_worker        S     20  0
2         1      gen_pop        S   5000  0
3         2          cow        S   1000  0
4         2  farm_worker        S     20  0
5         2      gen_pop        S   5000  0
6         3          cow        S     50  0
7         3  farm_worker        S      5  0
8         3      gen_pop        S  50000  0


## Cattle Migration and Population Dynamics

Now let's create a stable cattle population where cattle are born into farm 1 and then distributed to the other farms where they eventually die. We will consider this in a way that leads to a stable overall population of cattle in each location when the average life span of a dairy cow is 6 years (recognizing that in reality this process is more episodic as cow lifespans are limited by culling). 